<a href="https://colab.research.google.com/github/edgaracabral/PoDHackaton02_2023/blob/main/PoD%20FW%20Hackaton_02%2001%20DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto end-to-end do Super Hackaton

  Neste projeto vamos resolver o desafio do Super Hackaton, para isso faremos todas as etapas do processo de desenvolvimento de um modelo seguindo a metodologia CRISP-DM.

### **Introdução**

[descrever]

### **Avaliação do Modelo (Sugerido pelo Cliente)**
Métrica de Pontuação
As submissões são avaliadas usando o **Coeficiente Gini Normalizado**.

O Coeficiente Gini Normalizado ajusta a pontuação pelo máximo teórico, de modo que a pontuação máxima é 1.
___

# 1) Entendimento do problema de Negócio
   Construir um **modelo que preveja a probabilidade de um motorista fazer um sinistro de seguro auto no próximo ano**

[desenvolver]

# 2) Entendimento dos Dados

Para este projeto utilizaremos os dados disponibilizados pela seguradora Porto Seguro no ambiente Kaggle:

https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/data

[desenvolver]



# 3) Preparação dos Dados (DataPrep)
- Gerar Metadados da ABT (Tabela Analítica de Modelagem)
- Tratamento de missing (nulos)
 - Média para variáveis numéricas
 - 'Desconhecido' para variáveis categóricas
- Tratamento de categóricas de alta cardinalidade (LabelEncoder)
- Tratamento de categóricas de baixa cardinalidade (OneHotEncoder)
- Aplicar normalização a toda tabela de modelagem (ABT)
- Gerar artefatos para implantação do data prep realizado

## Carregando Google Drive e bibliotecas utilizadas no projeto

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


`pod_academy_functions` é a nossa biblioteca criada no curso de DS

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

# Redirecione para o folder on pod_academy_functions.py is localizado
%cd /content/drive/MyDrive/PoD Academy/modelos/Hackaton_DS_2023/PoD Framework
import pod_academy_functions as pod


/content/drive/MyDrive/PoD Academy/modelos/Hackaton_DS_2023/PoD Framework


## Lendo os dados disponibilizados

In [50]:
# Vá para o folder de dados
%cd /content/drive/MyDrive/PoD Academy/dados/Kaggle/Hackaton_DS_2023

df_train_00 = pd.read_csv('application_train.csv')
df_train_00.shape

/content/drive/MyDrive/PoD Academy/dados/Kaggle/Hackaton_DS_2023


(215257, 172)

In [51]:
# Checar Frequencia nas variáveis categóricas
print(pod.pod_custom_lowfreq(df_train_00))

                        Coluna            Indice  Frequencia
7           NAME_FAMILY_STATUS           Unknown           1
5             NAME_INCOME_TYPE   Maternity leave           2
1                  CODE_GENDER               XNA           3
11           ORGANIZATION_TYPE  Industry: type 8          21
6          NAME_EDUCATION_TYPE   Academic degree         117
4              NAME_TYPE_SUITE   Group of people         190
9              OCCUPATION_TYPE          IT staff         360
8            NAME_HOUSING_TYPE   Co-op apartment         796
13              HOUSETYPE_MODE    terraced house         847
14          WALLSMATERIAL_MODE            Others        1129
15         EMERGENCYSTATE_MODE               Yes        1627
12          FONDKAPREMONT_MODE  org spec account        3931
10  WEEKDAY_APPR_PROCESS_START            SUNDAY       11271
0           NAME_CONTRACT_TYPE   Revolving loans       20594
3              FLAG_OWN_REALTY                 N       65738
2                 FLAG_O

In [52]:
# Imprimir a frequência de cada valor na coluna
print(df_train_00['NAME_FAMILY_STATUS'].value_counts())

Married                 137750
Single / not married     31782
Civil marriage           20780
Separated                13745
Widow                    11199
Unknown                      1
Name: NAME_FAMILY_STATUS, dtype: int64


Redirecionado para pasta `dataprep`

In [53]:
# A partir de agora, todos os artefatos gerados serão colocados dentro da pasta dataprep
%cd /content/drive/MyDrive/PoD Academy/dados/Kaggle/Hackaton_DS_2023/dataprep

/content/drive/MyDrive/PoD Academy/dados/Kaggle/Hackaton_DS_2023/dataprep


## Renomeando ID e Target do original para o padrão do Framework

In [54]:
# Renomeie as colunas de id e target para o padrão do Framework (id, target)
df_train_00.rename(columns={'SK_ID_CURR': 'id', 'TARGET': 'target'}, inplace=True)

## Amostragem da base principal
- Serve também para testar se OneHot Encoding e Label Encoding estão tratando excessões

In [55]:
#df_train_00_sample = df_train_00.sample(n=100000, random_state=42)
df_train_00_sample = df_train_00
df_train_00_sample.shape

(215257, 172)

In [56]:
df_train_00_sample

,id,target,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
0,247330,0,Cash loans,F,N,N,0,157500.0,706410.0,67072.5,...,0.824762,0.333516,0.293260,0.564878,0.115058,0.655605,0.415562,0.092643,0.723331,0.796523
1,425716,1,Cash loans,F,Y,Y,1,121500.0,545040.0,25407.0,...,0.416260,0.404293,0.137944,0.457971,0.303691,0.215059,0.838892,0.608335,0.585643,0.298456
2,331625,0,Cash loans,M,Y,Y,1,225000.0,942300.0,27679.5,...,0.037711,0.124465,0.091840,0.364601,0.978220,0.520309,0.594523,0.559650,0.361873,0.254804
3,455397,0,Revolving loans,F,N,Y,2,144000.0,180000.0,9000.0,...,0.784630,0.831403,0.210872,0.049639,0.814219,0.830179,0.755163,0.216664,0.603002,0.429001
4,449114,0,Cash loans,F,N,Y,0,112500.0,729792.0,37390.5,...,0.265381,0.655344,0.668705,0.171391,0.335702,0.585494,0.619551,0.686738,0.540449,0.343632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215252,239009,0,Cash loans,F,N,Y,0,121500.0,675000.0,32602.5,...,0.214615,0.915096,0.760514,0.720699,0.875196,0.206345,0.359596,0.937269,0.699642,0.231518
215253,399937,0,Cash loans,F,N,N,2,225000.0,509922.0,40419.0,...,0.103347,0.958918,0.271508,0.308291,0.566643,0.942430,0.771174,0.495740,0.451506,0.601316
215254,253016,0,Cash loans,M,Y,Y,0,450000.0,1125000.0,33025.5,...,0.572026,0.709654,0.523142,0.424053,0.269128,0.789032,0.725667,0.160264,0.250138,0.573689
215255,270275,0,Revolving loans,M,Y,Y,1,180000.0,270000.0,13500.0,...,0.421224,0.175427,0.932756,0.585846,0.671286,0.519467,0.849646,0.485528,0.161590,0.091153


## Separando dados para garantir validação cruzada Holdout 70/30

In [57]:
# Suponha que você queira separar 70% dos dados para treino e 30% para validação
train, test = train_test_split(df_train_00_sample, test_size=0.3, random_state=42)
train.shape,test.shape

((150679, 172), (64578, 172))

In [58]:
train.head()

,id,target,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
45499,102669,0,Cash loans,F,N,Y,0,157500.0,709033.5,39721.5,...,0.614907,0.182116,0.283521,0.716671,0.041032,0.432577,0.302302,0.812673,0.622103,0.290383
74186,202196,1,Cash loans,F,N,Y,1,189000.0,640080.0,31261.5,...,0.855122,0.907023,0.555958,0.114399,0.959646,0.551736,0.793345,0.769783,0.029523,0.605118
65253,272854,0,Cash loans,F,N,N,1,121500.0,104256.0,8194.5,...,0.837285,0.401049,0.896599,0.775044,0.372050,0.355883,0.259412,0.282194,0.438214,0.348245
60400,207628,0,Cash loans,F,N,N,0,112500.0,755190.0,36328.5,...,0.046936,0.164097,0.416916,0.498222,0.366917,0.326498,0.383481,0.743987,0.612620,0.627862
71140,244369,1,Cash loans,F,N,N,1,193500.0,521280.0,25209.0,...,0.245522,0.720827,0.837383,0.520593,0.111493,0.763497,0.666572,0.421767,0.714451,0.916040


In [59]:
# Criando um novo dataframe baseado no original (Original fazia transformações aqui)
df_train_01 = train.copy()
df_test_01 = test.copy()

In [60]:
df_train_01.head()

,id,target,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
45499,102669,0,Cash loans,F,N,Y,0,157500.0,709033.5,39721.5,...,0.614907,0.182116,0.283521,0.716671,0.041032,0.432577,0.302302,0.812673,0.622103,0.290383
74186,202196,1,Cash loans,F,N,Y,1,189000.0,640080.0,31261.5,...,0.855122,0.907023,0.555958,0.114399,0.959646,0.551736,0.793345,0.769783,0.029523,0.605118
65253,272854,0,Cash loans,F,N,N,1,121500.0,104256.0,8194.5,...,0.837285,0.401049,0.896599,0.775044,0.372050,0.355883,0.259412,0.282194,0.438214,0.348245
60400,207628,0,Cash loans,F,N,N,0,112500.0,755190.0,36328.5,...,0.046936,0.164097,0.416916,0.498222,0.366917,0.326498,0.383481,0.743987,0.612620,0.627862
71140,244369,1,Cash loans,F,N,N,1,193500.0,521280.0,25209.0,...,0.245522,0.720827,0.837383,0.520593,0.111493,0.763497,0.666572,0.421767,0.714451,0.916040


In [61]:
metadados = pod.pod_academy_generate_metadata(df_train_01,
                                          ids=['id'],
                                          targets=['target'],
                                          orderby = 'PC_NULOS')

metadados

,FEATURE,USO_FEATURE,QT_NULOS,PC_NULOS,CARDINALIDADE,TIPO_FEATURE
0,COMMONAREA_MODE,Explicativa,105323,69.90,2681,float64
1,COMMONAREA_AVG,Explicativa,105323,69.90,2721,float64
2,COMMONAREA_MEDI,Explicativa,105323,69.90,2741,float64
3,NONLIVINGAPARTMENTS_AVG,Explicativa,104662,69.46,321,float64
4,NONLIVINGAPARTMENTS_MEDI,Explicativa,104662,69.46,178,float64
...,...,...,...,...,...,...
167,REG_CITY_NOT_WORK_CITY,Explicativa,0,0.00,2,int64
168,REG_CITY_NOT_LIVE_CITY,Explicativa,0,0.00,2,int64
169,LIVE_REGION_NOT_WORK_REGION,Explicativa,0,0.00,2,int64
170,REG_REGION_NOT_WORK_REGION,Explicativa,0,0.00,2,int64


## Excluindo variáveis com mais que 70% de nulos

In [62]:
missing_cutoff = 70

drop_vars_nulos = metadados[(metadados['PC_NULOS'] >= missing_cutoff)]
lista_drop_vars = list(drop_vars_nulos.FEATURE.values)

print('Variáveis que serão excluídas por alto percentual de nulos: ',lista_drop_vars)
# retirando lista de variáveis com alto percentual de nulos
df_train_02 = df_train_01.drop(axis=1,columns=lista_drop_vars)
df_train_02.shape

Variáveis que serão excluídas por alto percentual de nulos:  []


(150679, 172)

In [63]:
# Salvar a lista em um arquivo .pkl
with open('prd_drop_nullvars_hktn.pkl', 'wb') as f:
    pickle.dump(lista_drop_vars, f)

Tratando a base de teste da mesma forma que a de treino

In [64]:
# retirando lista de variáveis com alto percentual de nulos
df_test_02 = df_test_01.drop(axis=1,columns=lista_drop_vars)
df_test_02.shape

(64578, 172)

In [65]:
df_train_02.head()

,id,target,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
45499,102669,0,Cash loans,F,N,Y,0,157500.0,709033.5,39721.5,...,0.614907,0.182116,0.283521,0.716671,0.041032,0.432577,0.302302,0.812673,0.622103,0.290383
74186,202196,1,Cash loans,F,N,Y,1,189000.0,640080.0,31261.5,...,0.855122,0.907023,0.555958,0.114399,0.959646,0.551736,0.793345,0.769783,0.029523,0.605118
65253,272854,0,Cash loans,F,N,N,1,121500.0,104256.0,8194.5,...,0.837285,0.401049,0.896599,0.775044,0.372050,0.355883,0.259412,0.282194,0.438214,0.348245
60400,207628,0,Cash loans,F,N,N,0,112500.0,755190.0,36328.5,...,0.046936,0.164097,0.416916,0.498222,0.366917,0.326498,0.383481,0.743987,0.612620,0.627862
71140,244369,1,Cash loans,F,N,N,1,193500.0,521280.0,25209.0,...,0.245522,0.720827,0.837383,0.520593,0.111493,0.763497,0.666572,0.421767,0.714451,0.916040


In [66]:
# Retirar ID e Target do tratamento de nulos na base de treino
df_train_02 = df_train_02.drop(axis=1, columns=['id', 'target'])
df_train_02.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
45499,Cash loans,F,N,Y,0,157500.0,709033.5,39721.5,657000.0,Unaccompanied,...,0.614907,0.182116,0.283521,0.716671,0.041032,0.432577,0.302302,0.812673,0.622103,0.290383
74186,Cash loans,F,N,Y,1,189000.0,640080.0,31261.5,450000.0,Unaccompanied,...,0.855122,0.907023,0.555958,0.114399,0.959646,0.551736,0.793345,0.769783,0.029523,0.605118
65253,Cash loans,F,N,N,1,121500.0,104256.0,8194.5,90000.0,Unaccompanied,...,0.837285,0.401049,0.896599,0.775044,0.372050,0.355883,0.259412,0.282194,0.438214,0.348245
60400,Cash loans,F,N,N,0,112500.0,755190.0,36328.5,675000.0,Unaccompanied,...,0.046936,0.164097,0.416916,0.498222,0.366917,0.326498,0.383481,0.743987,0.612620,0.627862
71140,Cash loans,F,N,N,1,193500.0,521280.0,25209.0,450000.0,Unaccompanied,...,0.245522,0.720827,0.837383,0.520593,0.111493,0.763497,0.666572,0.421767,0.714451,0.916040


In [67]:
# Retirar ID e Target do tratamento de nulos na base de teste
df_test_02 = df_test_02.drop(axis=1, columns=['id', 'target'])
df_test_02.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
196348,Revolving loans,M,N,Y,0,90000.0,180000.0,9000.0,180000.0,Unaccompanied,...,0.593565,0.789703,0.254486,0.969768,0.918087,0.939984,0.941608,0.053827,0.571771,0.186664
147976,Cash loans,F,N,N,0,225000.0,781920.0,42547.5,675000.0,Unaccompanied,...,0.346320,0.912834,0.910227,0.784684,0.485868,0.812601,0.425869,0.335348,0.878763,0.792996
52662,Cash loans,M,N,Y,1,112500.0,450000.0,21888.0,450000.0,Unaccompanied,...,0.973969,0.671805,0.428278,0.189641,0.765047,0.931675,0.944781,0.270274,0.296258,0.518378
101577,Cash loans,F,N,Y,0,225000.0,760225.5,32337.0,679500.0,Unaccompanied,...,0.766011,0.533777,0.995612,0.998489,0.318540,0.923508,0.271327,0.558144,0.926976,0.794804
173078,Cash loans,F,N,N,0,225000.0,808650.0,26217.0,675000.0,Family,...,0.608260,0.155219,0.634449,0.650114,0.198636,0.202310,0.193130,0.437779,0.901853,0.126509


## Substituindo os nulos
- pela média para variáveis numéricas
- por 'POD_VERIFICAR" para categóricas

In [68]:
# Substituindo os nulos na base de treino
df_train_03, means = pod.pod_custom_fillna(df_train_02)

with open('prd_fillna_hktn.pkl', 'wb') as f:
  pickle.dump(means, f)

In [69]:
with open('prd_fillna_hktn.pkl', 'rb') as f:
  loaded_means = pickle.load(f)
loaded_means

{'CNT_CHILDREN': 0.4162092926021542,
 'AMT_INCOME_TOTAL': 168699.04549346626,
 'AMT_CREDIT': 599845.5172917261,
 'AMT_ANNUITY': 27149.62152063395,
 'AMT_GOODS_PRICE': 539122.5637288766,
 'REGION_POPULATION_RELATIVE': 0.020843482310076378,
 'DAYS_BIRTH': -16035.914852102815,
 'DAYS_EMPLOYED': 63599.31629036754,
 'DAYS_REGISTRATION': -4976.067520465273,
 'DAYS_ID_PUBLISH': -2991.8822818756307,
 'OWN_CAR_AGE': 12.035873743060138,
 'FLAG_MOBIL': 0.9999933633751219,
 'FLAG_EMP_PHONE': 0.8204792970486929,
 'FLAG_WORK_PHONE': 0.19891292084497508,
 'FLAG_CONT_MOBILE': 0.9981550182839015,
 'FLAG_PHONE': 0.2809283310879419,
 'FLAG_EMAIL': 0.05709488382588151,
 'CNT_FAM_MEMBERS': 2.153631229302026,
 'REGION_RATING_CLIENT': 2.0532987343956357,
 'REGION_RATING_CLIENT_W_CITY': 2.0322141771580644,
 'HOUR_APPR_PROCESS_START': 12.05614584646832,
 'REG_REGION_NOT_LIVE_REGION': 0.01548988246537341,
 'REG_REGION_NOT_WORK_REGION': 0.05142720618002509,
 'LIVE_REGION_NOT_WORK_REGION': 0.04101434174636147,
 '

In [70]:
# Substituindo os nulos na base de teste usando as médias da base de treino
df_test_03 = pod.pod_custom_fillna_prod(df_test_02, loaded_means)
df_test_03.shape

(64578, 170)

In [71]:
df_test_03.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
196348,Revolving loans,M,N,Y,0,90000.0,180000.0,9000.0,180000.0,Unaccompanied,...,0.593565,0.789703,0.254486,0.969768,0.918087,0.939984,0.941608,0.053827,0.571771,0.186664
147976,Cash loans,F,N,N,0,225000.0,781920.0,42547.5,675000.0,Unaccompanied,...,0.346320,0.912834,0.910227,0.784684,0.485868,0.812601,0.425869,0.335348,0.878763,0.792996
52662,Cash loans,M,N,Y,1,112500.0,450000.0,21888.0,450000.0,Unaccompanied,...,0.973969,0.671805,0.428278,0.189641,0.765047,0.931675,0.944781,0.270274,0.296258,0.518378
101577,Cash loans,F,N,Y,0,225000.0,760225.5,32337.0,679500.0,Unaccompanied,...,0.766011,0.533777,0.995612,0.998489,0.318540,0.923508,0.271327,0.558144,0.926976,0.794804
173078,Cash loans,F,N,N,0,225000.0,808650.0,26217.0,675000.0,Family,...,0.608260,0.155219,0.634449,0.650114,0.198636,0.202310,0.193130,0.437779,0.901853,0.126509


## Tratamento de categóricas de alta cardinalidade (LabelEncoder)

In [72]:
# Identifica colunas para LabelEncoder que não estejam na lista lista_drop_vars
card_cutoff = 30
df_categ_labelenc = metadados[(metadados['CARDINALIDADE'] > card_cutoff) & (metadados['TIPO_FEATURE'] == 'object')]
lista_vars_abt = list(df_train_03.columns)
lista_lenc = list(df_categ_labelenc.FEATURE.values)

for item in lista_drop_vars:
    if item in lista_lenc:
        lista_lenc.remove(item)

print('Lista de vars para Label Encoding: ',lista_lenc)

Lista de vars para Label Encoding:  ['ORGANIZATION_TYPE']


In [73]:
# transforma via LabelEncoder colunas na base de treino
import pickle
from sklearn.preprocessing import LabelEncoder

encoders = {}

for col in lista_lenc:
    encoder = LabelEncoder()
    df_train_03[col] = encoder.fit_transform(df_train_03[col])

    # Armazena o encoder para a coluna atual em um dicionário
    encoders[col] = encoder

# Salva o dicionário de encoders e a lista de colunas em um arquivo .pkl
data_to_serialize = {
    'encoders': encoders,
    'columns': lista_lenc
}

with open('prd_labelenc_hktn.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)

In [74]:
print(data_to_serialize)

{'encoders': {'ORGANIZATION_TYPE': LabelEncoder()}, 'columns': ['ORGANIZATION_TYPE']}


In [75]:
# Carregar os encoders e a lista de colunas
with open('prd_labelenc_hktn.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoders = loaded_data['encoders']
loaded_columns = loaded_data['columns']

# Suponha df_test_03 como sua base de teste
for col in loaded_columns:
    if col in loaded_encoders:
        # Transforma a coluna usando o encoder carregado
        df_test_03[col] = loaded_encoders[col].transform(df_test_03[col])


## Tratamento para categóricas de baixa cardinalidade

In [76]:
print(df_train_03.shape)
print(df_test_03.shape)

(150679, 170)
(64578, 170)


In [77]:
import pickle
from sklearn.preprocessing import OneHotEncoder

card_cutoff = 30
df_categ_onehot = metadados[(metadados['CARDINALIDADE'] <= card_cutoff) & (metadados['TIPO_FEATURE'] == 'object')]
lista_onehot = list(df_categ_onehot.FEATURE.values)
print('Lista de vars para OneHot Encoding: ',lista_onehot)

# Instanciando o encoder
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# Aplicando o one-hot encoding
encoded_data = encoder.fit_transform(df_train_03[lista_onehot])
encoded_cols = encoder.get_feature_names_out(lista_onehot)
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=df_train_03.index)

df_train_03 = pd.concat([df_train_03.drop(lista_onehot, axis=1), encoded_df], axis=1)

# Salva o encoder e a lista de colunas em um arquivo .pkl
data_to_serialize = {
    'encoder': encoder,
    'columns': lista_onehot
}

with open('prd_onehotenc_hktn.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)

df_train_03.shape

Lista de vars para OneHot Encoding:  ['FONDKAPREMONT_MODE', 'WALLSMATERIAL_MODE', 'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE', 'OCCUPATION_TYPE', 'NAME_TYPE_SUITE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_HOUSING_TYPE', 'NAME_FAMILY_STATUS', 'NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE', 'FLAG_OWN_REALTY', 'FLAG_OWN_CAR', 'CODE_GENDER', 'NAME_CONTRACT_TYPE']


(150679, 227)

In [78]:
# Imprimir a frequência de cada valor na coluna
print(df_test_03['NAME_FAMILY_STATUS'].value_counts())

Married                 41294
Single / not married     9639
Civil marriage           6122
Separated                4179
Widow                    3343
Unknown                     1
Name: NAME_FAMILY_STATUS, dtype: int64


In [79]:
# Carregar o encoder e a lista de colunas
with open('prd_onehotenc_hktn.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoder = loaded_data['encoder']
loaded_columns = loaded_data['columns']

# Suponha df_test_03 como sua base de teste
encoded_data_test = loaded_encoder.transform(df_test_03[loaded_columns])
encoded_cols_test = loaded_encoder.get_feature_names_out(loaded_columns)
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoded_cols_test, index=df_test_03.index)

df_test_03 = pd.concat([df_test_03.drop(loaded_columns, axis=1), encoded_df_test], axis=1)

df_test_03.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


(64578, 227)

## Aplicar padronização a toda tabela de modelagem tratada ate este ponto

In [80]:
import pickle
from sklearn.preprocessing import StandardScaler

# Excluindo IDs e Targets
df_id_target = metadados[(metadados['USO_FEATURE'] == 'ID') | (metadados['USO_FEATURE'] == 'Target')]
lista_id_target = list(df_id_target.FEATURE.values)
print('Lista de IDs e Target: ',lista_id_target)

# Instanciando o scaler
scaler = StandardScaler()

# Selecionando colunas numéricas
numeric_cols = df_train_03.select_dtypes(include=['float64', 'int64','int32']).columns

# Aplicando a normalização
df_train_03[numeric_cols] = scaler.fit_transform(df_train_03[numeric_cols])

# Salva o scaler em um arquivo .pkl
with open('prd_scaler_hktn.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print(scaler)
df_train_03.shape

Lista de IDs e Target:  ['id', 'target']
StandardScaler()


(150679, 227)

In [81]:
# Carregar o scaler
with open('prd_scaler_hktn.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

# Suponha df_test_03 como sua base de teste

# Selecionando colunas numéricas
numeric_cols = df_test_03.select_dtypes(include=['float64', 'int64','int32']).columns

# Aplicando a normalização
df_test_03[numeric_cols] = loaded_scaler.fit_transform(df_test_03[numeric_cols])

df_test_03.shape

(64578, 227)

In [82]:
df_test_03.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,FLAG_OWN_REALTY_Y,FLAG_OWN_CAR_Y,CODE_GENDER_M,CODE_GENDER_XNA,NAME_CONTRACT_TYPE_Revolving loans
196348,-0.581516,-0.784592,-1.038705,-1.246965,-0.968061,0.749096,1.473657,-0.456240,-1.187740,1.214111,...,-0.469724,-0.276030,-0.009639,-0.006816,-1.032745,0.663684,-0.716515,1.388779,-0.005565,3.070985
147976,-0.581516,0.569449,0.454599,1.070650,0.369944,-0.153229,-0.942629,-0.476321,-1.494607,-0.315301,...,-0.469724,-0.276030,-0.009639,-0.006816,-1.032745,-1.506741,-0.716515,-0.720057,-0.005565,-0.325628
52662,0.810888,-0.558918,-0.368861,-0.356603,-0.238240,-0.089271,1.002376,-0.455399,1.357274,-0.532746,...,-0.469724,-0.276030,-0.009639,-0.006816,0.968293,0.663684,-0.716515,1.388779,-0.005565,-0.325628
101577,-0.581516,0.569449,0.400777,0.365262,0.382107,-0.979424,1.369436,-0.453802,-0.252929,1.534976,...,-0.469724,-0.276030,-0.009639,-0.006816,0.968293,0.663684,-0.716515,-0.720057,-0.005565,-0.325628
173078,-0.581516,0.569449,0.520914,-0.057536,0.369944,-1.060308,-0.099489,-0.512637,-0.989412,1.983789,...,-0.469724,3.622799,-0.009639,-0.006816,-1.032745,-1.506741,-0.716515,-0.720057,-0.005565,-0.325628


In [83]:
## Trazer o id e target para a tabela pós dataprep

abt_train = df_train_03.merge(train[['id','target']], left_index=True, right_index=True, how='inner')
abt_test = df_test_03.merge(test[['id','target']], left_index=True, right_index=True, how='inner')

print(abt_train.shape)
print(abt_test.shape)

(150679, 229)
(64578, 229)


In [84]:
abt_train.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,FLAG_OWN_REALTY_Y,FLAG_OWN_CAR_Y,CODE_GENDER_M,CODE_GENDER_XNA,NAME_CONTRACT_TYPE_Revolving loans,id,target
45499,-0.577795,-0.103326,0.271059,0.864546,0.319009,0.272835,0.998359,-0.460894,1.003091,-0.761866,...,-0.007287,-0.009289,-1.032565,0.662808,-0.72079,-0.722077,-0.002576,-0.3251,102669,0
74186,0.810437,0.187303,0.099882,0.282767,-0.241190,1.838198,0.822512,-0.451879,0.959652,0.936079,...,-0.007287,-0.009289,0.968462,0.662808,-0.72079,-0.722077,-0.002576,-0.3251,202196,1
65253,0.810437,-0.435473,-1.230301,-1.303511,-1.215451,1.082825,1.418004,-0.453537,1.177699,0.318825,...,-0.007287,-0.009289,0.968462,-1.508732,-0.72079,-0.722077,-0.002576,-0.3251,272854,0
60400,-0.577795,-0.518510,0.385643,0.631216,0.367722,-0.783151,1.561712,-0.456982,1.318237,1.637533,...,-0.007287,-0.009289,0.968462,-1.508732,-0.72079,-0.722077,-0.002576,-0.3251,207628,0
71140,0.810437,0.228821,-0.195039,-0.133453,-0.241190,-0.043280,0.191667,-0.466323,0.604191,-0.521860,...,-0.007287,-0.009289,-1.032565,-1.508732,-0.72079,-0.722077,-0.002576,-0.3251,244369,1


## Salvando tabelas de treino e teste pós preparação dos dados

In [85]:
abt_train.to_csv('abt_train.csv')
abt_test.to_csv('abt_test.csv')